In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sn
import folium
import math
import json
from branca.colormap import LinearColormap

In [2]:
intercountry_df = pd.read_csv('../Data/output_intercountry_tone_count_table', dtype={'EventRootCode': np.str})
european_borders = json.load(open('../Data/europe_borders.topojson.json'))
world_borders = json.load(open('../Data/world_borders.topojson.json'))

In [3]:
intercountry_df = intercountry_df[pd.notnull(intercountry_df['LocEvent'])]
intercountry_df = (intercountry_df[intercountry_df["EventRootCode"]!='--'])

In [4]:
#europe_coordinates = [54.5260, 15.2551]
#europe_map = folium.Map(europe_coordinates, tiles='Mapbox bright', zoom_start=5.5)

# Defining a style function to color the map and the borders
#style_func=lambda feature: {'fillColor': '#990033','color' : 'black','weight' : 2 }

# Using a GeoJsonTooltip to display cantons' name when mouse is over them
#tool=folium.GeoJsonTooltip(fields=['geounit'],aliases=[''], sticky=True, style="font-family: Arial; color: black;", \
#                           opacity=0.8, direction='top')

# Load border information on the map
#folium.TopoJson(european_borders, object_path='objects.continent_Europe_subunits', style_function=style_func, tooltip=tool, overlay=False) \
#    .add_to(europe_map)
#europe_map

In [5]:
world_map = folium.Map(tiles='Mapbox bright', zoom_start=5.5)

# Defining a style function to color the map and the borders
style_func=lambda feature: {'fillColor': '#990033','color' : 'black','weight' : 2 }

# Using a GeoJsonTooltip to display cantons' name when mouse is over them
tool=folium.GeoJsonTooltip(fields=['name'],aliases=['Country:'], sticky=True, style="font-family: Arial; color: black;", \
                          opacity=0.8, direction='top')

# Load border information on the map
folium.TopoJson(world_borders, object_path='objects.countries1', style_function=style_func, tooltip=tool, overlay=False) \
    .add_to(world_map)
world_map


In [6]:
#intercountry_df['EventRootCode'].value_counts()

In [7]:
fips_codes = pd.read_csv('../Data/fipscountries.csv')
fips_codes = fips_codes[['fips','country']]
#eu_fips_codes = fips_codes[fips_codes['continent']=='EU']
#eu_fips_codes = eu_fips_codes[['fips','country']]

In [8]:
intercountry_df = fips_codes.merge(intercountry_df, left_on='fips', right_on='LocMention', how='inner')
intercountry_df.head()
#United States of America in the topojson file
intercountry_df.loc[intercountry_df['country']=='United States', 'country'] = 'United States of America'
intercountry_df.loc[intercountry_df['country']=='Serbia', 'country'] = 'Republic of Serbia'
#intercountry_df['country'].unique()

In [9]:
intercountry_df = intercountry_df[['country', 'AverageTone']]
intercountry_df = intercountry_df.groupby('country', as_index=False).mean()
intercountry_df.head()

,country,AverageTone
0,Afghanistan,-3.552023
1,Albania,-3.165164
2,Algeria,-2.827580
3,American Samoa,1.739897
4,Andorra,-1.044770


In [10]:
intercountry_df_dict = intercountry_df.set_index('country')['AverageTone'].to_dict()

In [11]:
#color_scale = LinearColormap(['yellow','red'], vmin = min(intercountry_df['AverageTone']), 
 #                                              vmax = max(intercountry_df['AverageTone'])
  #                          )

color_scale = LinearColormap(['yellow','red'], vmin = min(intercountry_df_dict.values()), \
                                               vmax = max(intercountry_df_dict.values()) \
                            )

#def color_function(country, df, column, color_fun):
def color_function(feature):
    #value = intercountry_df_dict.get(feature['properties']['geounit']) Europe
    value = intercountry_df_dict.get(feature['properties']['name'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_scale(value)

In [12]:
folium.TopoJson(
    european_borders,
    object_path='objects.continent_Europe_subunits',
    style_function=lambda feature: {
        #'fillColor': color_function(feature['properties']['geounit'], intercountry_df, 'AverageTone', color_scale),
        'fillColor': color_function(feature),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5',
        'fillOpacity' : 0.8
        },
    tooltip=folium.GeoJsonTooltip(fields=['geounit'],aliases=['Country: '], sticky=True, style="font-family: Arial; color: black;", \
                           opacity=0.8, direction='top'),
    name = 'Country Media Average',
    ).add_to(europe_map)

color_scale.caption = 'Average Tone'
europe_map.add_child(color_scale)
folium.LayerControl().add_to(europe_map)
europe_map

NameError: name 'europe_map' is not defined

In [13]:
folium.TopoJson(
    world_borders,
    object_path='objects.countries1',
    style_function=lambda feature: {
        #'fillColor': color_function(feature['properties']['geounit'], intercountry_df, 'AverageTone', color_scale),
        'fillColor': color_function(feature),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5',
        'fillOpacity' : 0.8
        },
    tooltip=folium.GeoJsonTooltip(fields=['name'],aliases=['Country: '], sticky=True, style="font-family: Arial; color: black;", \
                           opacity=0.8, direction='top'),
    name = 'Country Media Average',
    ).add_to(world_map)

color_scale.caption = 'Average Tone'
world_map.add_child(color_scale)
folium.LayerControl().add_to(world_map)
world_map

In [ ]:
intercountry_df.groupby('LocMention')['EventRootCode'].value_counts().unstack(level=-1).sort_values(by=['19'], ascending = False)


In [ ]:
#intercountry_df.plot(x='LocMention', y='NumberOfInterMentions', kind='pie')

#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(norm_sparse_eigenvecs[:,1].real, norm_sparse_eigenvecs[:,2].real, norm_sparse_eigenvecs[:,0].real, c=norm_kmeans, cmap='cool')

In [ ]:
Syria_mentions = intercountry_df[intercountry_df['LocEvent']=='SY']

In [ ]:
a = Syria_mentions[(Syria_mentions['EventRootCode']=='19')]#.sort_values(by=['NumberOfInterMentions'], ascending = False)

In [ ]:
a.head()

In [ ]:
a[a['AverageTone']<-6]